In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.neighbors import NearestNeighbors
from bramm_data_analysis import loading
import geopandas as gpd
import numpy as np

### Load Moss Data

In [ ]:
columns = {}

columns["metadata"] = [
    "site_code",
    "site_insee_code",
    "department_code",
    "latitude",
    "longitude",
    "x_lambert",
    "y_lambert",
    "altitude",
    "date",
    "sample_code",
    "sample_send_europe",
]
columns["sample"] = [
    "strand_size",
    "visible_dust_particles",
    "visible_pollen_particles",
    "mineral_type",
]
columns["site"] = [
    "samples_nb",
    "fern_samples_nb",
    "tree_samples_nb",
    "herb_samples_nb",
    "litter_samples_nb",
    "humus_samples_nb",
    "soi_samples_nb",
    "sand_samples_nb",
    "hard_strain_coniferous_samples_nb",
    "hard_strain_hardwood_samples_nb",
    "hard_strain_unknown_samples_nb",
    "decomposed_strain_coniferous_samples_nb",
    "decomposed_strain_hardwood_samples_nb",
    "decomposed_strain_unknown_samples_nb",
    "bark_coniferous_nb",
    "bark_hardwood_samples_nb",
    "bark_unknown_samples_nb",
    "hard_coniferous_samples_nb",
    "hard_hardwood_samples_nb",
    "hard_unknown_samples_nb",
    "decomposed_coniferous_samples_nb",
    "decomposed_hardwood_samples_nb",
    "decomposed_unknown_samples_nb",
]
columns["studies"] = [
    "sample_outside_complementary_study",
    "cs_3_species_comparison",
    "cs_2_species_comparison",
    "cs_repeated_sampling",
    "cs_repeated_analysis",
]
columns["incertitudes"] = [
    "aluminium_incertitude",
    "arsenic_incertitude",
    "calcium_incertitude",
    "cadmium_incertitude",
    "cobalt_incertitude",
    "chromium_incertitude",
    "copper_incertitude",
    "iron_incertitude",
    "mercury_incertitude",
    "nitrogen_incertitude",
    "sodium_incertitude",
    "nickel_incertitude",
    "lead_incertitude",
    "sulfur_incertitude",
    "zinc_incertitude",
]
columns["categorical"] = [
    "weather",
    "tree_layer",
    "tree_layer_complement",
    "tree_cover",
    "species",
]
df_sites = loading.load_sites(Path("../data/Mines_2024.xlsx"))
df_samples = loading.load_samples(Path("../data/Mines_2024.xlsx"))
df_values = loading.load_values(Path("../data/Mines_2024.xlsx"))
df_moss = pd.merge(
    pd.merge(
        df_sites,
        df_samples,
        on="site_code",
    ),
    df_values,
    on="sample_code",
)


# Correct data
for col in [
    "sodium",
    "platinium",
    "rhodium",
    "antimony",
    "strontium",
    "vanadium",
    "zinc",
]:
    df_moss[col] = df_moss[col].astype(str)
    df_moss[col] = df_moss[col].apply(lambda x: x.replace(",", "."))
    df_moss[col] = df_moss[col].apply(lambda x: x.replace("< ", ""))
    df_moss[col].astype("float64")

In [ ]:
df_rmqs = pd.read_csv("../data/RMQS.csv")

In [ ]:
estimator = NearestNeighbors(n_neighbors=1, metric="haversine")
moss_xy = df_moss[["longitude", "latitude"]]
rmqs_xy = df_rmqs[["longitude", "latitude"]]
estimator.fit(rmqs_xy)
dist, indexes = estimator.kneighbors(moss_xy)

In [ ]:
rmqs_match = df_rmqs.loc[indexes.flatten(), :]
rmqs_match.index = moss_xy.index
df = pd.concat([df_moss, rmqs_match], axis=1)

In [ ]:
gdf_moss = gpd.GeoDataFrame(
    data=df_moss,
    geometry=gpd.points_from_xy(
        x=df_moss["longitude"], y=df_moss["latitude"], crs="epsg:4326"
    ),
)
gdf_rmqs_match = gpd.GeoDataFrame(
    data=rmqs_match,
    geometry=gpd.points_from_xy(
        x=rmqs_match["longitude"], y=rmqs_match["latitude"], crs="epsg:4326"
    ),
)
gdf_rmqs = gpd.GeoDataFrame(
    data=df_rmqs,
    geometry=gpd.points_from_xy(
        x=df_rmqs["longitude"], y=df_rmqs["latitude"], crs="epsg:4326"
    ),
)

In [ ]:
gdf_moss.plot()

In [ ]:
gdf_rmqs_match.plot()

In [ ]:
gdf_rmqs.plot()